# BoxRetriever

这将帮助你开始使用 Box [检索器](/docs/concepts/retrievers)。关于 BoxRetriever 功能和配置的详细文档，请访问 [API 参考](https://python.langchain.com/api_reference/box/retrievers/langchain_box.retrievers.box.BoxRetriever.html)。

# 概述

`BoxRetriever` 类可帮助你以 Langchain 的 `Document` 格式从 Box 获取非结构化内容。你可以通过基于全文搜索来查找文件，或者使用 Box AI 来检索包含针对文件执行的 AI 查询结果的 `Document`。这需要包含一个 `List[str]`，其中包含 Box 文件 ID，例如 `["12345","67890"]`

:::info
Box AI 需要企业版 Plus 许可证
:::

没有文本表示的文件将被跳过。

### 集成详情

1：自带数据（即索引和搜索自定义文档语料库）：

| Retriever | Self-host | Cloud offering | Package |
| :--- | :--- | :---: | :---: |
[BoxRetriever](https://python.langchain.com/api_reference/box/retrievers/langchain_box.retrievers.box.BoxRetriever.html) | ❌ | ✅ | langchain-box |

## 设置

要使用 Box 包，你需要准备以下几项：

* 一个 Box 账户 — 如果你目前不是 Box 用户，或者想在生产环境外的 Box 实例中进行测试，可以使用 [免费开发者账户](https://account.box.com/signup/n/developer#ty9l3)。
* [一个 Box 应用](https://developer.box.com/guides/getting-started/first-application/) — 这个应用在 [开发者控制台](https://account.box.com/developers/console) 中配置，并且必须启用 `Manage AI` 范围才能使用 Box AI。你还可以在这里选择你的身份验证方法。
* 该应用必须 [由管理员启用](https://developer.box.com/guides/authorization/custom-app-approval/#manual-approval)。对于免费开发者账户，这是注册该账户的用户。

### 凭据

在这些示例中，我们将使用 [令牌身份验证](https://developer.box.com/guides/authentication/tokens/developer-tokens)。这可以与任何 [身份验证方法](https://developer.box.com/guides/authentication/) 一起使用。只需使用任何方法获取令牌即可。如果你想了解如何将其他身份验证类型与 `langchain-box` 结合使用，请访问 [Box 提供商](/docs/integrations/providers/box) 文档。

In [2]:
import getpass
import os

box_developer_token = getpass.getpass("Enter your Box Developer Token: ")

如果你想从单个查询中获得自动追踪，你也可以通过取消注释以下内容来设置你的 [LangSmith](https://docs.smith.langchain.com/) API 密钥：

In [3]:
# os.environ["LANGSMITH_API_KEY"] = getpass.getpass("Enter your LangSmith API key: ")
# os.environ["LANGSMITH_TRACING"] = "true"

### 安装

此检索器存在于 `langchain-box` 包中：

In [4]:
%pip install -qU langchain-box

Note: you may need to restart the kernel to use updated packages.


## 实例化

现在我们可以实例化我们的检索器了：

## 搜索

In [5]:
from langchain_box import BoxRetriever

retriever = BoxRetriever(box_developer_token=box_developer_token)

为了进行更精细化的搜索，我们提供了一系列选项来帮助您筛选结果。这会结合使用 `langchain_box.utilities.SearchOptions`、`langchain_box.utilities.SearchTypeFilter` 以及 `langchain_box.utilities.DocumentFiles` 枚举，根据创建日期、文件搜索部分，甚至限定搜索范围到特定文件夹进行筛选。

更多信息，请查看 [API 参考文档](https://python.langchain.com/v0.2/api_reference/box/utilities/langchain_box.utilities.box.SearchOptions.html)。

In [6]:
from langchain_box.utilities import BoxSearchOptions, DocumentFiles, SearchTypeFilter

box_folder_id = "260931903795"

box_search_options = BoxSearchOptions(
    ancestor_folder_ids=[box_folder_id],
    search_type_filter=[SearchTypeFilter.FILE_CONTENT],
    created_date_range=["2023-01-01T00:00:00-07:00", "2024-08-01T00:00:00-07:00,"],
    k=200,
    size_range=[1, 1000000],
    updated_data_range=None,
)

retriever = BoxRetriever(
    box_developer_token=box_developer_token, box_search_options=box_search_options
)

retriever.invoke("AstroTech Solutions")

[Document(metadata={'source': 'https://dl.boxcloud.com/api/2.0/internal_files/1514555423624/versions/1663171610024/representations/extracted_text/content/', 'title': 'Invoice-A5555_txt'}, page_content='Vendor: AstroTech Solutions\nInvoice Number: A5555\n\nLine Items:\n    - Gravitational Wave Detector Kit: $800\n    - Exoplanet Terrarium: $120\nTotal: $920')]

## Box AI

In [7]:
from langchain_box import BoxRetriever

box_file_ids = ["1514555423624", "1514553902288"]

retriever = BoxRetriever(
    box_developer_token=box_developer_token, box_file_ids=box_file_ids
)

## 用法

In [8]:
query = "What was the most expensive item purchased"

retriever.invoke(query)

[Document(metadata={'source': 'Box AI', 'title': 'Box AI What was the most expensive item purchased'}, page_content='The most expensive item purchased is the **Gravitational Wave Detector Kit** from AstroTech Solutions, which costs **$800**.')]

## 引用

借助 Box AI 和 `BoxRetriever`，您可以返回提示的答案、Box 用于获取该答案的引用，或者两者都返回。无论您如何选择使用 Box AI，检索器都会返回一个 `List[Document]` 对象。我们通过两个 `bool` 参数 `answer` 和 `citations` 提供这种灵活性。`answer` 默认为 `True`，`citations` 默认为 `False`，因此如果您只想获取答案，可以省略两者。如果您想同时获取两者，可以仅包含 `citations=True`；如果您只想获取引用，则应包含 `answer=False` 和 `citations=True`。

### 同时获取两者

In [11]:
retriever = BoxRetriever(
    box_developer_token=box_developer_token, box_file_ids=box_file_ids, citations=True
)

retriever.invoke(query)

[Document(metadata={'source': 'Box AI', 'title': 'Box AI What was the most expensive item purchased'}, page_content='The most expensive item purchased is the **Gravitational Wave Detector Kit** from AstroTech Solutions, which costs **$800**.'),
 Document(metadata={'source': 'Box AI What was the most expensive item purchased', 'file_name': 'Invoice-A5555.txt', 'file_id': '1514555423624', 'file_type': 'file'}, page_content='Vendor: AstroTech Solutions\nInvoice Number: A5555\n\nLine Items:\n    - Gravitational Wave Detector Kit: $800\n    - Exoplanet Terrarium: $120\nTotal: $920')]

### 仅引用

In [12]:
retriever = BoxRetriever(
    box_developer_token=box_developer_token,
    box_file_ids=box_file_ids,
    answer=False,
    citations=True,
)

retriever.invoke(query)

[Document(metadata={'source': 'Box AI What was the most expensive item purchased', 'file_name': 'Invoice-A5555.txt', 'file_id': '1514555423624', 'file_type': 'file'}, page_content='Vendor: AstroTech Solutions\nInvoice Number: A5555\n\nLine Items:\n    - Gravitational Wave Detector Kit: $800\n    - Exoplanet Terrarium: $120\nTotal: $920')]

## 在链中使用

与其它检索器一样，BoxRetriever 可以通过[链](/docs/how_to/sequence/)集成到 LLM 应用中。

我们需要一个 LLM 或聊天模型：

import ChatModelTabs from "@theme/ChatModelTabs";

<ChatModelTabs customVarName="llm" />

In [7]:
openai_key = getpass.getpass("Enter your OpenAI key: ")

Enter your OpenAI key:  ········


In [8]:
# | output: false
# | echo: false

from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0, openai_api_key=openai_key)

In [42]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

box_search_options = BoxSearchOptions(
    ancestor_folder_ids=[box_folder_id],
    search_type_filter=[SearchTypeFilter.FILE_CONTENT],
    created_date_range=["2023-01-01T00:00:00-07:00", "2024-08-01T00:00:00-07:00,"],
    k=200,
    size_range=[1, 1000000],
    updated_data_range=None,
)

retriever = BoxRetriever(
    box_developer_token=box_developer_token, box_search_options=box_search_options
)

context = "You are a finance professional that handles invoices and purchase orders."
question = "Show me all the items purchased from AstroTech Solutions"

prompt = ChatPromptTemplate.from_template(
    """Answer the question based only on the context provided.

    Context: {context}

    Question: {question}"""
)


def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [43]:
chain.invoke(question)

'- Gravitational Wave Detector Kit: $800\n- Exoplanet Terrarium: $120'

## 用作代理工具

与其他的检索器一样，`BoxRetriever` 也可以作为工具添加到 LangGraph 代理中。

In [ ]:
pip install -U langsmith

In [27]:
from langchain import hub
from langchain.agents import AgentExecutor, create_openai_tools_agent
from langchain.tools.retriever import create_retriever_tool

In [34]:
box_search_options = BoxSearchOptions(
    ancestor_folder_ids=[box_folder_id],
    search_type_filter=[SearchTypeFilter.FILE_CONTENT],
    created_date_range=["2023-01-01T00:00:00-07:00", "2024-08-01T00:00:00-07:00,"],
    k=200,
    size_range=[1, 1000000],
    updated_data_range=None,
)

retriever = BoxRetriever(
    box_developer_token=box_developer_token, box_search_options=box_search_options
)

box_search_tool = create_retriever_tool(
    retriever,
    "box_search_tool",
    "This tool is used to search Box and retrieve documents that match the search criteria",
)
tools = [box_search_tool]

In [35]:
prompt = hub.pull("hwchase17/openai-tools-agent")
prompt.messages

llm = ChatOpenAI(temperature=0, openai_api_key=openai_key)

agent = create_openai_tools_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools)

/Users/shurrey/local/langchain/.venv/lib/python3.11/site-packages/langsmith/client.py:312: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [36]:
result = agent_executor.invoke(
    {
        "input": "list the items I purchased from AstroTech Solutions from most expensive to least expensive"
    }
)

In [37]:
print(f"result {result['output']}")

result The items you purchased from AstroTech Solutions from most expensive to least expensive are:

1. Gravitational Wave Detector Kit: $800
2. Exoplanet Terrarium: $120

Total: $920


## 额外字段

所有 Box 连接器都能够从 Box `FileFull` 对象中选择额外的字段，作为自定义 LangChain 元数据返回。每个对象都接受一个可选的 `List[str]`，名为 `extra_fields`，其中包含返回对象中的 json 键，例如 `extra_fields=["shared_link"]`。

该连接器会将此字段添加到集成运行所需字段的列表中，然后将结果添加到 `Document` 或 `Blob` 中返回的元数据里，例如 `"metadata" : { "source" : "source， "shared_link" : "shared_link" }`。如果该文件没有该字段，则返回一个空字符串，例如 `"shared_link" : ""`。

## API 参考

有关 BoxRetriever 所有功能和配置的详细文档，请前往 [API 参考](https://python.langchain.com/api_reference/box/retrievers/langchain_box.retrievers.box.BoxRetriever.html)。


## 帮助

如果您有任何疑问，可以查看我们的 [开发者文档](https://developer.box.com) 或通过我们的 [开发者社区](https://community.box.com) 与我们联系。